In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import cv2
import glob

In [ ]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [ ]:
Autoencoder = tf.keras.models.load_model('../input/model-final/autoencoder-model_justCSA5.h5')
Autoencoder.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(Autoencoder, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
import cv2
import glob

path = '../input/test-set/Test_normal_data_update'
path_list = []
evl_list =[]
keypoints_with_scores_train = []
for k in glob.glob(os.path.join(path,'*.mp4')):
    frame_list = []
    print(k)
    path_list.append(k)
    cap = cv2.VideoCapture(k)
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
    
    # Resize image
        img = frame.copy()
        img = img[680:2000, 400:680]
        #cv2.imwrite('/kaggle',img)
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 224,224)
        input_img = tf.cast(img, dtype=tf.int32)
    
       
    # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,0,:51].reshape(51)
        frame_list.append(keypoints_with_scores)
        frame_list_np = np.asarray(frame_list)
    keypoints_with_scores_train.append(frame_list_np[30:120,:])
    keypoints_with_scores_train_np = np.asarray(keypoints_with_scores_train)
    print(keypoints_with_scores_train_np.shape)
    evl = Autoencoder.evaluate(keypoints_with_scores_train_np,keypoints_with_scores_train_np)
    evl_list.append(evl)

In [ ]:
df1=pd.DataFrame(evl_list,columns=['loss','mae'])
df1['file'] = path_list
#df1[df1['loss']>0.0130].count()

In [ ]:
df2=pd.DataFrame(evl_list,columns=['loss'])
df2['file'] = path_list
df2[df2['loss']>0.0130].count()

In [ ]:
df2.to_csv("normal_csv")

In [ ]:
df1.to_csv("N_modeljustCSAfinal.csv")

In [ ]:
import cv2
import glob

path = '../input/csa-normal-walk-updated/normal walk CSA'
path_list = []
evl_list =[]
for k in glob.glob(os.path.join(path,'*.mp4')):
    frame_list = []
    print(k)
    path_list.append(k)
    cap = cv2.VideoCapture(k)
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
    
    # Resize image
        img = frame.copy()
        img = img[680:2000, 400:680]
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 224,224)
        input_img = tf.cast(img, dtype=tf.int32)
    
       
    # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,0,:51].reshape(51)
        frame_list.append(keypoints_with_scores)
        frame_list_np = np.asarray(frame_list)
    keypoints_with_scores_train = []
    for x in range(90,len(frame_list_np)):
        keypoints_with_scores_train.append(frame_list_np[x-90:x,:])
    keypoints_with_scores_train = np.asarray(keypoints_with_scores_train)
    print(keypoints_with_scores_train.shape)
    evl = Autoencoder.evaluate(keypoints_with_scores_train,keypoints_with_scores_train)
    evl_list.append(evl)

In [ ]:
df3=pd.DataFrame(evl_list,columns=['mse','mae','cosin'])
df3['file'] = path_list
df3

In [ ]:
df3.to_csv("normal_video_withmodel4.csv")

In [ ]:
import cv2
import glob

path = '../input/highly-drunk/highly drunk walk'
path_list = []
evl_list =[]
for k in glob.glob(os.path.join(path,'*.mp4')):
    frame_list = []
    print(k)
    path_list.append(k)
    cap = cv2.VideoCapture(k)
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
    
    # Resize image
        img = frame.copy()
        img = img[680:2000, 400:680]
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 224,224)
        input_img = tf.cast(img, dtype=tf.int32)
    
       
    # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,0,:51].reshape(51)
        frame_list.append(keypoints_with_scores)
        frame_list_np = np.asarray(frame_list)
    keypoints_with_scores_train = []
    for x in range(90,len(frame_list_np)):
        keypoints_with_scores_train.append(frame_list_np[x-90:x,:])
    keypoints_with_scores_train = np.asarray(keypoints_with_scores_train)
    print(keypoints_with_scores_train.shape)
    evl = Autoencoder.evaluate(keypoints_with_scores_train,keypoints_with_scores_train)
    evl_list.append(evl)

In [ ]:
df4=pd.DataFrame(evl_list,columns=['mse','mae','cosin'])
df4['file'] = path_list
df4

In [ ]:
df4.to_csv("normal_video_csv.csv")

In [ ]:
df4.to_csv("highly+drunk_video_withmodel4.csv")